# 02 — AfriSenti Multilingual Modeling (XLM‑R / AfriBERTa vs LSTM)
Implements tasks 19–24: preprocessing, tokenization, fine‑tuning, evaluation, attention viz, ablations, and cross‑lingual testing.

## 0. Setup & Imports

In [ ]:

import os, re, json, random, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path
import torch, evaluate
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import label_binarize
from datasets import Dataset as HFDataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
import emoji
import torch
import evaluate

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ROOT = Path.cwd().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd()
DATA_DIR = ROOT/'data'; FIG_DIR = ROOT/'outputs'/'figures'; TAB_DIR = ROOT/'outputs'/'tables'
os.makedirs(FIG_DIR, exist_ok=True); os.makedirs(TAB_DIR, exist_ok=True)
print('Device:', DEVICE)


ModuleNotFoundError: No module named 'torch'

## 1. Load Data (local CSVs or HuggingFace)

In [2]:

import pandas as pd
def load_local_csvs(data_dir: Path):
    csvs = list(data_dir.glob('*.csv'))
    frames = []
    for p in csvs:
        try: df = pd.read_csv(p)
        except UnicodeDecodeError: df = pd.read_csv(p, encoding='utf-8-sig')
        except Exception: df = pd.read_csv(p, sep='\t')
        cols = {c.lower(): c for c in df.columns}
        text_col = cols.get('text') or cols.get('tweet') or list(df.columns)[0]
        label_col = cols.get('label') or cols.get('sentiment')
        lang_col  = cols.get('lang') or cols.get('language')
        tmp = pd.DataFrame()
        tmp['text'] = df[text_col].astype(str)
        tmp['label'] = df[label_col] if label_col is not None else pd.NA
        if lang_col: tmp['lang'] = df[lang_col].astype(str)
        else:
            name = p.stem.lower()
            tmp['lang'] = 'sw' if 'sw' in name else ('am' if 'am' in name else ('en' if 'en' in name else 'unk'))
        frames.append(tmp[['text','label','lang']])
    return pd.concat(frames, ignore_index=True) if frames else None

# Prefer local CSVs; optionally replace with HF loader
df = load_local_csvs(DATA_DIR)
if df is None:
    # fallback tiny sample for structure testing only
    df = pd.DataFrame({
        'text':['Nimefurahi sana!','This is terrible...','እዚ በጣም ጥሩ ነው።','Sio mbaya','Not good','Average only'],
        'label':['positive','negative','positive','neutral','negative','neutral'],
        'lang':['sw','en','am','sw','en','en']
    })
    print('Using toy sample; add CSVs in data/ for real training.')
df = df.dropna(subset=['text']).copy()
df['text'] = df['text'].astype(str).str.replace('\s+',' ',regex=True).str.strip()
df['label'] = df['label'].astype(str).str.lower()
df['lang']  = df['lang'].astype(str).str.lower()
df.head()


NameError: name 'DATA_DIR' is not defined

## 2. Preprocessing (URLs, mentions, hashtags, emojis, slang) — Task 19

In [ ]:

URL_RE = re.compile(r'https?://\S+|www\.\S+', re.I)
MENT_RE = re.compile(r'@\w+')
HASH_RE = re.compile(r'#(\w+)')
WS_RE   = re.compile(r'\s+')

SLANG_MAP = {
    'en': {'u':'you','ur':'your','btw':'by the way','idk':'I do not know'},
    'sw': {'btw':'by the way'},  # extend as needed
    'am': {}
}

def normalize_slang(text, lang):
    dic = SLANG_MAP.get(lang, {})
    toks = text.split()
    toks = [dic.get(t.lower(), t) for t in toks]
    return ' '.join(toks)

def clean_text(text, lang):
    text = URL_RE.sub(' URL ', text)
    text = MENT_RE.sub(' @USER ', text)
    text = HASH_RE.sub(lambda m: f'HASHTAG_{m.group(1)}', text)
    text = emoji.demojize(text, language='en')  # 🙂 -> :slightly_smiling_face:
    text = normalize_slang(text, lang)
    text = WS_RE.sub(' ', text).strip()
    return text

df['clean_text'] = [clean_text(t,l) for t,l in zip(df['text'], df['lang'])]
df[['lang','label','text','clean_text']].head()


## 3. Split + Label Mapping

In [ ]:

valid = {'negative','neutral','positive'}
df = df[df['label'].isin(valid)].reset_index(drop=True)
label2id = {'negative':0, 'neutral':1, 'positive':2}
id2label = {v:k for k,v in label2id.items()}
df['label_id'] = df['label'].map(label2id)

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=SEED, stratify=df[['lang','label']])
train_df, val_df  = train_test_split(train_df, test_size=0.1765, random_state=SEED, stratify=train_df[['lang','label']])  # ~0.15 val

print('Splits:', train_df.shape, val_df.shape, test_df.shape)


## 4. Tokenization (XLM‑R default; switch to AfriBERTa) — Task 19

In [ ]:

MODEL_NAME = 'xlm-roberta-base'   # or: 'castorini/afriberta_base'
MAX_LEN = 128
BATCH_SIZE = 16
LR = 2e-5
EPOCHS = 3  # set to 5 if you want

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tok_fn(batch):
    return tokenizer(batch['clean_text'], padding='max_length', truncation=True, max_length=MAX_LEN)

train_tok = HFDataset.from_pandas(train_df[['clean_text','label_id','lang']]).map(tok_fn, batched=True, remove_columns=['clean_text'])
val_tok   = HFDataset.from_pandas(val_df[['clean_text','label_id','lang']]).map(tok_fn, batched=True, remove_columns=['clean_text'])
test_tok  = HFDataset.from_pandas(test_df[['clean_text','label_id','lang']]).map(tok_fn, batched=True, remove_columns=['clean_text'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


## 5. Fine‑tune Transformer with Early Stopping & Gradient Clipping — Tasks 20–21

In [ ]:

acc = evaluate.load('accuracy')
f1  = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    a = acc.compute(predictions=preds, references=labels)['accuracy']
    f = f1.compute(predictions=preds, references=labels, average='macro')['f1']
    # ROC-AUC (one-vs-rest)
    probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
    yb = label_binarize(labels, classes=[0,1,2])
    try:
        auc = roc_auc_score(yb, probs, average='macro', multi_class='ovr')
    except Exception:
        auc = float('nan')
    return {'accuracy': a, 'f1_macro': f, 'roc_auc_ovr': auc}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3, id2label=id2label, label2id=label2id
).to(DEVICE)

args = TrainingArguments(
    output_dir=str(ROOT/'outputs'/'transformers_runs'),
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro',
    greater_is_better=True,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    warmup_ratio=0.06,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    max_grad_norm=1.0  # gradient clipping
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()
print('Validation (best):', trainer.evaluate())


## 6. Final Evaluation on Test Set — Task 22

In [ ]:

pred = trainer.predict(test_tok)
logits = pred.predictions
y_true = np.array(test_tok['label_id'])
y_pred = logits.argmax(axis=-1)
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()

acc_val = (y_pred == y_true).mean()
f1_macro = f1.compute(predictions=y_pred, references=y_true, average='macro')['f1']
yb = label_binarize(y_true, classes=[0,1,2])
try:
    auc = roc_auc_score(yb, probs, average='macro', multi_class='ovr')
except Exception:
    auc = float('nan')

print('Test Accuracy:', acc_val, 'Macro-F1:', f1_macro, 'ROC-AUC(OVR):', auc)
print(classification_report(y_true, y_pred, target_names=[id2label[i] for i in [0,1,2]]))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[id2label[i] for i in [0,1,2]],
            yticklabels=[id2label[i] for i in [0,1,2]])
plt.title('Confusion Matrix — Transformer')
plt.tight_layout(); plt.savefig(FIG_DIR/'afrisenti_confusion_transformer.png', dpi=150); plt.show()

# Example predictions
for i in np.random.choice(len(test_df), size=min(5, len(test_df)), replace=False):
    print('—'*40)
    print('Text:', test_df.iloc[i]['text'])
    print('Clean:', test_df.iloc[i]['clean_text'])
    print('Lang:', test_df.iloc[i]['lang'], '| Label:', test_df.iloc[i]['label'])
    print('Pred:', id2label[int(y_pred[i])], '| Prob:', probs[i].round(3))


## 7. Attention Visualization (single layer/head) — Task 22

In [ ]:

model.eval()
sample_txt = test_df.iloc[0]['clean_text'] if len(test_df) > 0 else 'sample text'
enc = tokenizer(sample_txt, return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_LEN)
enc = {k: v.to(DEVICE) for k,v in enc.items()}
with torch.no_grad():
    out = model(**enc, output_attentions=True)
atts = out.attentions  # tuple: (layers) x (batch, heads, seq, seq)
if atts:
    att = atts[-1][0, 0].detach().cpu().numpy()  # last layer, head 0
    plt.figure(figsize=(5,4))
    plt.imshow(att[:50,:50], aspect='auto', cmap='viridis')
    plt.colorbar(); plt.title('Attention heatmap (last layer, head 0)')
    plt.tight_layout(); plt.savefig(FIG_DIR/'attention_heatmap_sample.png', dpi=150); plt.show()
else:
    print('Model did not return attentions.')


## 8. LSTM Baseline — Task 20 (comparison)

In [ ]:

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class LSTMDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df['clean_text'].tolist()
        self.labels = df['label_id'].tolist()
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tok(self.texts[idx], truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        item = {k: v.squeeze(0) for k,v in enc.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128, num_classes=3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=tokenizer.pad_token_id or 0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim*2, num_classes)
    def forward(self, input_ids, attention_mask=None, labels=None):
        x = self.embedding(input_ids)
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        last = self.dropout(last)
        logits = self.fc(last)
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {'loss': loss, 'logits': logits}

train_loader = DataLoader(LSTMDataset(train_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(LSTMDataset(val_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE)
test_loader  = DataLoader(LSTMDataset(test_df, tokenizer, MAX_LEN), batch_size=BATCH_SIZE)

lstm = LSTMClassifier(vocab_size=tokenizer.vocab_size).to(DEVICE)
opt = torch.optim.Adam(lstm.parameters(), lr=1e-3)
best_val = float('inf'); patience = 2; wait = 0

for epoch in range(5):  # up to 5 epochs
    lstm.train(); tot=0; n=0
    for b in train_loader:
        ids = b['input_ids'].to(DEVICE); labels = b['labels'].to(DEVICE)
        opt.zero_grad()
        out = lstm(input_ids=ids, labels=labels)
        torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1.0)
        out['loss'].backward(); opt.step()
        tot += float(out['loss']); n += 1
    lstm.eval(); vtot=0; vn=0
    with torch.no_grad():
        for b in val_loader:
            ids = b['input_ids'].to(DEVICE); labels = b['labels'].to(DEVICE)
            out = lstm(input_ids=ids, labels=labels); vtot += float(out['loss']); vn += 1
    print(f'Epoch {epoch+1}: train_loss={tot/n:.4f}, val_loss={vtot/vn:.4f}')
    if vtot/vn < best_val:
        best_val = vtot/vn; wait = 0; torch.save(lstm.state_dict(), ROOT/'outputs'/'lstm_best.pt')
    else:
        wait += 1
        if wait >= patience:
            print('Early stopping.'); break

# Test LSTM
lstm.load_state_dict(torch.load(ROOT/'outputs'/'lstm_best.pt', map_location=DEVICE)); lstm.eval()
yt, yp, pp = [], [], []
with torch.no_grad():
    for b in test_loader:
        ids = b['input_ids'].to(DEVICE); labels = b['labels'].to(DEVICE)
        logits = lstm(input_ids=ids)['logits']
        preds = logits.argmax(dim=-1)
        yt.extend(labels.cpu().numpy().tolist())
        yp.extend(preds.cpu().numpy().tolist())
        pp.extend(torch.softmax(logits, dim=-1).cpu().numpy().tolist())

yt = np.array(yt); yp = np.array(yp); pp = np.array(pp)
print('LSTM Test Accuracy:', (yp==yt).mean())
print(classification_report(yt, yp, target_names=[id2label[i] for i in [0,1,2]]))


## 9. Ablations — Task 23 (subset for speed)

In [ ]:

def small_subset(df, n=1000):
    if len(df) <= n: return df
    return df.sample(n=n, random_state=SEED).reset_index(drop=True)

def run_ablation(bs_list=[8,16], lr_list=[2e-5,5e-5], max_len_list=[64,128]):
    results = []
    base_tr = small_subset(train_df, n=1000)
    base_va = small_subset(val_df, n=400)
    for bs in bs_list:
        for lr in lr_list:
            for ml in max_len_list:
                tok = AutoTokenizer.from_pretrained(MODEL_NAME)
                def tfn(b): return tok(b['clean_text'], padding='max_length', truncation=True, max_length=ml)
                tr = HFDataset.from_pandas(base_tr[['clean_text','label_id']]).map(tfn, batched=True, remove_columns=['clean_text'])
                va = HFDataset.from_pandas(base_va[['clean_text','label_id']]).map(tfn, batched=True, remove_columns=['clean_text'])
                m  = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(DEVICE)
                a  = TrainingArguments(output_dir=str(ROOT/'outputs'/'ablations'), evaluation_strategy='epoch', save_strategy='no',
                                       per_device_train_batch_size=bs, per_device_eval_batch_size=bs,
                                       num_train_epochs=1, learning_rate=lr, logging_steps=100, fp16=torch.cuda.is_available())
                t  = Trainer(model=m, args=a, train_dataset=tr, eval_dataset=va, tokenizer=tok,
                             data_collator=DataCollatorWithPadding(tok),
                             compute_metrics=lambda p: {'f1_macro': evaluate.load('f1').compute(predictions=p.predictions.argmax(-1), references=p.label_ids, average='macro')['f1']})
                t.train()
                ev = t.evaluate()
                results.append({'batch_size': bs, 'lr': lr, 'max_len': ml, 'f1_macro': ev['eval_f1_macro']})
                print('ABL:', results[-1])
    df_res = pd.DataFrame(results)
    df_res.to_csv(TAB_DIR/'ablations_results.csv', index=False)
    return df_res

# To run (optional, can take time):
# ablation_table = run_ablation()
# ablation_table.head()


## 10. Cross‑Lingual Testing — Task 24

In [ ]:

def train_on_lang(train_lang='sw', test_lang='en', epochs=3):
    tr = train_df[train_df['lang']==train_lang]
    va = val_df[val_df['lang']==train_lang]
    te = test_df[test_df['lang']==test_lang]
    if len(tr)<10 or len(te)<10:
        print('Not enough data for selected languages.'); return None

    tok = AutoTokenizer.from_pretrained(MODEL_NAME)
    def tf(b): return tok(b['clean_text'], padding='max_length', truncation=True, max_length=MAX_LEN)

    tr_hf = HFDataset.from_pandas(tr[['clean_text','label_id']]).map(tf, batched=True, remove_columns=['clean_text'])
    va_hf = HFDataset.from_pandas(va[['clean_text','label_id']]).map(tf, batched=True, remove_columns=['clean_text'])
    te_hf = HFDataset.from_pandas(te[['clean_text','label_id']]).map(tf, batched=True, remove_columns=['clean_text'])

    m = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(DEVICE)
    a = TrainingArguments(output_dir=str(ROOT/'outputs'/'crosslingual'),
                          evaluation_strategy='epoch', save_strategy='no',
                          num_train_epochs=epochs, learning_rate=LR,
                          per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE,
                          fp16=torch.cuda.is_available())
    t = Trainer(model=m, args=a, train_dataset=tr_hf, eval_dataset=va_hf, tokenizer=tok,
                data_collator=DataCollatorWithPadding(tok), compute_metrics=compute_metrics)
    t.train()
    pred = t.predict(te_hf)
    y_true = np.array(te_hf['label_id']); y_pred = pred.predictions.argmax(-1)
    f1_macro = evaluate.load('f1').compute(predictions=y_pred, references=y_true, average='macro')['f1']
    acc = (y_pred==y_true).mean()
    print(f'Cross-lingual {train_lang}→{test_lang}: acc={acc:.4f}, f1_macro={f1_macro:.4f}')
    return {'train_lang':train_lang,'test_lang':test_lang,'acc':acc,'f1_macro':f1_macro}

# Example (uncomment when you have real data):
# cross1 = train_on_lang('sw', 'en', epochs=3)
# cross2 = train_on_lang('sw', 'am', epochs=3)
